In [1]:
! pip install transformers
! pip install datasets

     |████████████████████████████████| 2.6 MB 7.4 MB/s 
     |████████████████████████████████| 895 kB 50.3 MB/s 
     |████████████████████████████████| 636 kB 53.0 MB/s 
     |████████████████████████████████| 3.3 MB 47.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 6.6 MB/s 
     |████████████████████████████████| 243 kB 61.3 MB/s 
     |████████████████████████████████| 118 kB 74.7 MB/s 


In [2]:
import pandas as pd
import numpy as np
import pdb
import os
os.chdir('/content/drive/MyDrive/CAMemBERT2')
import re
import string
import copy
from preprocessing2 import LinkGedDatasetToEssayDataset,CreateHuggingFaceDictGed,CreateHuggingFaceMultiTask
from models import run_model
from transformers import (AutoModelForSequenceClassification,TrainingArguments,Trainer,
                          AutoTokenizer,AutoModelForTokenClassification,AutoModel,AutoConfig,
                          TrainerControl,TrainerCallback)
from transformers.models.bert.modeling_bert import TokenClassifierOutput
from datasets import DatasetDict,Dataset,load_dataset
from scipy.stats import spearmanr
from sklearn.metrics import cohen_kappa_score,classification_report
from math import floor
import torch 
from torch import nn
from collections import defaultdict


In [3]:
multi_task_obj = CreateHuggingFaceMultiTask()

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/content/drive/MyDrive/CAMemBERT2/preprocessing2.py:33: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','correct'])
/content/drive/MyDrive/CAMemBERT2/preprocessing2.py:33: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','correct'])
/content/drive/MyDrive/CAMemBERT2/preprocessing2.py:33: ParserWarning: Falling back 

...

  0%|          | 0/242 [00:00<?, ?ex/s]

  0%|          | 0/268 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [4]:
ged_class_weights = multi_task_obj.ged_obj.get_weights()
global ged_class_weights
truncated_input_ner = multi_task_obj.ner_obj.max_length_for_training
global truncated_input_ner

In [ ]:
class TaggingHead(nn.Module):

    def __init__(self,task_name,mini_task_dict,pretrained_model_name='distilroberta-base',shared_encoder_layer=None):
        super().__init__()
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        config = AutoConfig.from_pretrained(pretrained_model_name,
                                                        num_labels=mini_task_dict['n_labels'],
                                                        output_hidden_states=True)
        self.config = config
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels).to(self.device)
        self.class_weights = mini_task_dict['class_weights']
        self.loss_fct = nn.CrossEntropyLoss(weight = self.class_weights) if self.class_weights  is not None else nn.CrossEntropyLoss()
        self.shared_encoder_layer = shared_encoder_layer

    def set_shared_encoder_layer(self,encoder_layer):
        self.shared_encoder_layer = encoder_layer

    def forward(self,original_model_output,inputs):
        output = self.shared_encoder_layer(original_model_output).last_hidden_state if self.shared_encoder_layer else original_model_output
        sequence_output = self.dropout(output)
        logits = self.classifier(sequence_output)
        loss = None
        labels = inputs['labels']
        # Only keep active parts of the loss
        active_loss = inputs['attention_mask'].view(-1) == 1
        active_logits = logits.view(-1, self.config.num_labels)
        active_labels = torch.where(active_loss, labels.view(-1), torch.tensor(self.loss_fct.ignore_index).type_as(labels))
        non_padded_mask = active_labels!=-100
        loss = self.loss_fct(active_logits[non_padded_mask], active_labels[non_padded_mask])
        active_preds_mask = torch.logical_and(active_loss,non_padded_mask)
        return {'loss':loss,'preds':torch.argmax(logits,2).flatten()[active_preds_mask],'labels':active_labels[non_padded_mask]}

class MultiTaskModel(nn.Module):

    _task_dict = {
        'aes':{
            'alpha':1.0,'n_labels':1,'labels':'script_score',
            'class_weights':None,'head_type':'classification',
            'model':AutoModelForSequenceClassification,'shared_layer':False,
            'additional_truncation':False,'output_layer':-1
            },
        'ged':{
            'alpha':1.0,'n_labels':2,'labels':'labels',
            'class_weights':ged_class_weights,'head_type':'tagging',
            'model':AutoModelForTokenClassification,'shared_layer':1,
            'additional_truncation':False,'output_layer':-1
            },
        'ner':{
            'alpha':0.1,'n_labels':9,'labels':'labels',
            'class_weights':None,'head_type':'tagging',
            'model':AutoModelForTokenClassification,'shared_layer':1,
            'additional_truncation':truncated_input_ner,'output_layer':-1
            },
        }

    def __init__(self,pretrained_model='distilroberta-base',task_dict=None,current_task=None):
        super().__init__()
        self.task_dict = task_dict if task_dict!=None else self._task_dict
        new_tasks = set(self._task_dict.keys())-set(self.task_dict.keys())
        if len(new_tasks)>0:
            raise KeyError(f'task not yet supported {new_tasks}')
        self.model = self.task_dict['aes']['model'].from_pretrained(pretrained_model,
                                                               num_labels=self.task_dict['aes']['n_labels'],
                                                               output_hidden_states=True)
        self.shared_encoder_layer = None
        self.current_task = current_task
        self.decoder_dict = {}
        for task_name,mini_task_dict in self.task_dict.items():
            if task_name!='aes':
                if self.shared_encoder_layer==None and mini_task_dict['shared_layer']:
                    config = AutoConfig.from_pretrained(pretrained_model,num_hidden_layers=1,output_hidden_states=True)
                    shared_encoder = AutoModel.from_pretrained(pretrained_model,config=config).encoder
                    self.shared_encoder_layer = shared_encoder
                self.decoder_dict[task_name] = TaggingHead(task_name,mini_task_dict)
                self.decoder_dict[task_name].set_shared_encoder_layer(self.shared_encoder_layer)


    def forward(self,**inputs):
        if len(torch.unique(inputs['script_scores']))>1 and -100 in inputs['script_scores']:
            output_dict = {}
            loss = 0.0
            for inputs_by_task in self.split_inputs(inputs):
                task_name,inputs_for_task = inputs_by_task
                self.current_task = task_name
                outputs = self.get_outputs(inputs_for_task)
                loss += outputs.pop('loss')
                output_dict = {**output_dict,**outputs}
            output_dict['loss'] = loss
            return output_dict
        else:
            self.current_task = 'ner' if -100 in inputs['script_scores'] else 'aes_ged'
            return self.get_outputs(inputs)

    def get_outputs(self,inputs):
        if self.current_task=='aes_ged':
            model_output = self.model(input_ids=inputs['input_ids'],attention_mask=inputs['attention_mask'],labels=inputs['script_scores'].float())
            hidden_state_for_ged = model_output.hidden_states[self.task_dict['ged']['output_layer']]
            ged_output = self.decoder_dict['ged'](hidden_state_for_ged,inputs)
            return {'loss':model_output.loss + ged_output['loss'],'aes_loss':model_output.loss,'aes_preds':model_output.logits.flatten(),'aes_labels':inputs['script_scores'],
                    'ged_loss':ged_output['loss'],'ged_preds':ged_output['preds'],'ged_labels':ged_output['labels']}
        else:
            model_output = self.model(input_ids=inputs['input_ids'],attention_mask=inputs['attention_mask'])
            hidden_state_for_ner = model_output.hidden_states[self.task_dict['ner']['output_layer']]
            ner_output = self.decoder_dict['ner'](hidden_state_for_ner,inputs)
            return {'loss':ner_output['loss'],'ner_loss':ner_output['loss'],'ner_preds':ner_output['preds'],'ner_labels':ner_output['labels']}
        
    def split_inputs(self,inputs):
        split_mask_1 = inputs['script_scores']!=-100
        return (('aes_ged',{k:v[split_mask_1] for k,v in inputs.items()}),('ner',{k:v[~split_mask_1] for k,v in inputs.items()}))

# model = MultiTaskModel()

for name,params in model.model.named_parameters():
    if 'classifier' not in name:
        params.requires_grad = False

batch_size = 8
args = TrainingArguments(
    output_dir = '/content',
    save_total_limit = 1,
    evaluation_strategy = 'no',
    learning_rate= 5e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs =30,
    weight_decay=0.01,
    # load_best_model_at_end=True,
    # save_strategy = 'epoch',
    # metric_for_best_model='pearson',
)


class TrainerDos(Trainer):

    _map_for_hist = {'aes_loss':'aes','aes_preds':'aes','aes_labels':'aes','ged_loss':'ged','ged_preds':'ged','ged_labels':'ged','ner_loss':'ner','ner_preds':'ner','ner_labels':'ner'}
    _map_for_eval_cal = {'aes_loss':'loss','aes_preds':'preds','aes_labels':'labels','ged_loss':'loss','ged_preds':'preds','ged_labels':'labels','ner_loss':'loss','ner_preds':'preds','ner_labels':'labels'}
    prev_epoch = 0
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    temp = 1
    task_index = {'aes':0,'ged':1,'ner':2}
    opt_weight = 'dwa'

    def compute_loss(self,model,inputs,return_outputs = False):
        epoch = floor(self.state.epoch)
        step_in_epoch = self.state.global_step+1-(epoch*(self.state.max_steps/self.state.num_train_epochs))

        if self.state.global_step==0:
            self.avg_cost = torch.zeros([self.state.num_train_epochs, 6]).float().to(self.device)
            self.lambda_weight = torch.ones([3, self.state.num_train_epochs]).to(self.device)
        
        if step_in_epoch==1:
            if epoch == 0 or epoch == 1:
                self.lambda_weight[0, epoch] = 1.0
                self.lambda_weight[1, epoch] = 1.0
                self.lambda_weight[2, epoch] = 0.1
            else:
                T = self.temp
                w_1 = self.avg_cost[epoch - 1, 0] / self.avg_cost[epoch - 2, 0]
                w_2 = self.avg_cost[epoch - 1, 1] / self.avg_cost[epoch - 2, 1]
                w_3 = self.avg_cost[epoch - 1, 2] / self.avg_cost[epoch - 2, 2]
                self.lambda_weight[0, epoch] = 3 * torch.exp(w_1 / T) / (torch.exp(w_1 / T) + torch.exp(w_2 / T) + torch.exp(w_3 / T))
                self.lambda_weight[1, epoch] = 3 * torch.exp(w_2 / T) / (torch.exp(w_1 / T) + torch.exp(w_2 / T) + torch.exp(w_3 / T))
                self.lambda_weight[2, epoch] = 3 * torch.exp(w_3 / T) / (torch.exp(w_1 / T) + torch.exp(w_2 / T) + torch.exp(w_3 / T))
            self.cost = torch.zeros(3).float().to(self.device)

        outputs = self.model(**inputs)
        loss = outputs['loss']
        
        
        #################

        if self.opt_weight == 'dwa':
            if self.model.current_task == 'aes_ged':
                train_loss = [outputs['aes_loss'],outputs['ged_loss']]
                loss = sum([self.lambda_weight[i, epoch] * train_loss[i] for i in range(2)])
                self.cost[0] = train_loss[0].item()
                self.cost[1] = train_loss[1].item()
                self.avg_cost[epoch, :2] += self.cost[:2] / (self.state.max_steps/self.state.num_train_epochs)
            else:
                loss = self.lambda_weight[2, epoch] * outputs['ner_loss']
                self.cost[2] = outputs['ner_loss'].item()
                self.avg_cost[epoch, 2] += self.cost[2] / (self.state.max_steps/self.state.num_train_epochs)

        if (self.state.global_step+1)%(self.state.max_steps/self.state.num_train_epochs)==0:
            self.compute_metrics_2()
            self.prev_epoch = self.state.epoch

        ##################
        return loss


    ###################################
    def compute_metrics_2(self):
        history = {task:defaultdict(list) for task in self.model.task_dict.keys()}
        losses = defaultdict(int)
        total_loss = 0.0
        for step,inputs in enumerate(self.get_eval_dataloader(self.eval_dataset)):     
            for key, value in inputs.items():
                inputs[key] = inputs[key].to('cuda')
            outputs = self.model(**inputs)
            for key,value in outputs.items():
                if key == 'loss':
                    total_loss += value.item()
                elif 'loss' in key:
                    losses[key] += value.item()
                else:
                    history[self._map_for_hist[key]][self._map_for_eval_cal[key]] = history[self._map_for_hist[key]][self._map_for_eval_cal[key]] + value.cpu().detach().numpy().tolist()

        for k,v in losses.items():
            task = k.split('_')[0]
            print(f'{k} : {v/step}')
            print(f'{task}_weight_coef : {self.lambda_weight[self.task_index[task],floor(self.state.epoch)]}')
        print(f"loss_total:{total_loss/step}")
        for task,values in history.items():
            if task == 'aes':
                preds,labels = np.rint(values['preds']),values['labels']
                rmse = np.sqrt(np.mean((preds-labels)**2))
                pearson = np.corrcoef(preds,labels)[0,1]
                coef, p = spearmanr(preds, labels)
                kappa = cohen_kappa_score(preds,labels,weights='quadratic')
                print({
                    f"rmse_{task}": rmse,
                    f"pearson_{task}": pearson,
                    f"spearman_{task}" : coef,
                    f"kappa_{task}":kappa
                    })
            else:
                preds,labels = values['preds'],values['labels']
                digits = 9 if task == 'ner' else 2
                print(classification_report(labels, preds, digits=digits))
    ###################################
    

    def get_train_dataloader(self):
        train_dataset = self.train_dataset
        return torch.utils.data.DataLoader(
            train_dataset,
            batch_size=self.args.train_batch_size,
            collate_fn=self.data_collator,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers,
            pin_memory=self.args.dataloader_pin_memory,
        )

    def get_eval_dataloader(self,eval_dataset):
        return torch.utils.data.DataLoader(
            eval_dataset,
            batch_size=self.args.train_batch_size,
            collate_fn=self.data_collator,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers,
            pin_memory=self.args.dataloader_pin_memory,
        )

encoded_dataset = multi_task_obj.get_combined_dataset()


trainer = TrainerDos(
    model,
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['dev'],
    callbacks=[MyCallback]  # We can either pass the callback class this way or an instance of it (MyCallback())
)
trainer.forced_weights = [1.0,1.0,0.1]
trainer.range_before_updating = 5

#############
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 2122
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7980


Step,Training Loss
500,6.809800


aes_loss : 4.712398817372876
aes_weight_coef : 1.0
ged_loss : 0.15673286970271622
ged_weight_coef : 1.0
ner_loss : 0.12250664119803628
ner_weight_coef : 0.10000000149011612
loss_total:4.991638350452101
{'rmse_aes': 4.495831402532795, 'pearson_aes': 0.6506534429563444, 'spearman_aes': 0.6693686002137039, 'kappa_aes': 0.5905129953777374}
              precision    recall  f1-score   support

           0       0.89      0.94      0.92     28702
           1       0.42      0.27      0.33      4473

    accuracy                           0.85     33175
   macro avg       0.65      0.61      0.62     33175
weighted avg       0.83      0.85      0.84     33175

              precision    recall  f1-score   support

           0  0.987106118 0.988306555 0.987705972     42759
           1  0.932889628 0.913137894 0.922908093      1842
           2  0.968797565 0.973986228 0.971384968      1307
           3  0.813586098 0.768083520 0.790180284      1341
           4  0.827238335 0.873501997 0.

Saving model checkpoint to /content/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [/content/checkpoint-2000] due to args.save_total_limit


In [ ]:
trainer.control.should_evaluate

True

In [ ]:
dict_ = defaultdict(list)
dict_['loss'] = dict_['loss'] + lst_1

In [ ]:
dict_['loss']

[0, 1, 2]

In [ ]:
lst_1 = [0,1,2]
dict_['loss'] = dict_['loss'] + lst_1
dict_['loss']

[0, 1, 2]

In [ ]:
dict_['loss'] = dict_['loss'] + lst_1
dict_['loss']

[0, 1, 2, 0, 1, 2]

In [ ]:
conll2003 = load_dataset("conll2003")

Downloading:   0%|          | 0.00/2.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/650k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/146k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6. Subsequent calls will reuse this data.


In [ ]:
def append_sep(example):
    tokens,tags = example['tokens'],example['ner_tags']
    tokens.append(tokenizer.sep_token)
    tags.append(-100)
    return {'tokens':tokens,'ner_tags':tags}
conll2003 = conll2003.map(append_sep)

  0%|          | 0/14041 [00:00<?, ?ex/s]

  0%|          | 0/3250 [00:00<?, ?ex/s]

  0%|          | 0/3453 [00:00<?, ?ex/s]

In [ ]:
flattened_values = np.concatenate(conll2003['train']['tokens'])
split_arr = np.array_split(flattened_values,dataset['train'].num_rows)
max_len_row = max([len(row) for row in split_arr])
padded_array = []
for row in split_arr:
    padded_array.append(np.pad(row,(0,max_len_row-len(row)),constant_values = (tokenizer.pad_token,tokenizer.pad_token)))


In [ ]:
pd.DataFrame(padded_array)


TypeError: ignored

In [ ]:
np.pad(split_arr[-1],(0,len(split_arr[0])-len(split_arr[-1])),constant_values = (tokenizer.pad_token,tokenizer.pad_token))

array([')', '71', '77', '</s>', '70', ',', 'Jose', 'Coceres', '(',
       'Argentina', ')', '69', '78', '71', ',', 'Paul', 'Eales', '75',
       '71', '72', ',', '</s>', 'Klas', 'Eriksson', '(', 'Sweden', ')',
       '71', '75', '72', ',', 'Mike', 'Clayton', '(', 'Australia', ')',
       '</s>', '69', '76', '73', ',', 'Mark', 'Roe', '69', '71', '78',
       '</s>', '219', 'Eamonn', 'Darcy', '(', 'Ireland', ')', '74', '76',
       '69', ',', 'Bob', 'May', '(', 'U.S.', ')', '74', '75', '70', ',',
       '</s>', 'Paul', 'Lawrie', '72', '75', '72', ',', 'Miguel', 'Angel',
       'Jimenez', '(', 'Spain', ')', '74', '72', '</s>', '73', ',',
       'Peter', 'Mitchell', '74', '71', '75', ',', 'Philip', 'Walton',
       '(', 'Ireland', ')', '71', '74', '</s>', '74', ',', 'Peter',
       "O'Malley", '(', 'Australia', ')', '71', '73', '75', '</s>', '220',
       'Barry', 'Lane', '73', '77', '70', ',', 'Wayne', 'Riley', '(',
       'Australia', ')', '71', '78', '71', ',', '</s>', 'Martin', 'Gates'

In [ ]:
205*1061

217505

In [ ]:
len_flat

217662

In [ ]:
train = LinkGedDatasetToEssayDataset('train').get_updated_df()               
dev = LinkGedDatasetToEssayDataset('dev').get_updated_df()
test = LinkGedDatasetToEssayDataset('test').get_updated_df()
train['set_type'] = 'train'
test['set_type'] = 'test'
dev['set_type'] = 'dev'
df = pd.concat([train,test,dev],axis=0)
df = df.rename(columns={'essays':'text','script-s':'script_scores'})
df['script_scores'] = df['script_scores'].astype(int)        
df = df.groupby('id').agg({'text':list,'script_scores':list,'set_type':list})
df[ 'text' ] = df[ 'text' ].str.join(' ')
df[ 'script_scores' ] = df[ 'script_scores' ].apply(lambda x : x[0])
df[ 'set_type' ] = df[ 'set_type' ].apply(lambda x : x[0])
df = df.reset_index()[['text','script_scores','set_type']]
dataset_dict = {set_type:Dataset.from_pandas(df.groupby('set_type').get_group(set_type)) for set_type in ['train','test','dev']}

from transformers import AutoTokenizer

initial_checkpoint = "distilroberta-base"   
tokenizer = AutoTokenizer.from_pretrained(initial_checkpoint, use_fast=True)

def preprocess_function(examples):
    return tokenizer(examples['text'],truncation=True, padding = True, max_length = 512)

encoded_dataset = {set_type : dataset.map(preprocess_function, batched=True) for set_type,dataset in dataset_dict.items()}
columns_to_drop = list(set(encoded_dataset['train'].column_names) - {'attention_mask', 'input_ids', 'script_scores'})
encoded_dataset = {set_type : dataset.remove_columns(columns_to_drop).rename_column('script_scores','labels') for set_type,dataset in encoded_dataset.items()}

import numpy as np
from sklearn.metrics import accuracy_score
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from scipy.stats import spearmanr
import torch
import torch.nn.functional as F
import copy
from transformers import AutoModelForSequenceClassification

num_labels = 1
model = AutoModelForSequenceClassification.from_pretrained('distilroberta-base', num_labels=num_labels)

#freezing layers if needed
modules = [model.roberta.embeddings, *model.roberta.encoder.layer[:6]] #freeze all layers
for module in modules:
    for param in module.parameters():
        param.requires_grad = False

batch_size = 8


args = TrainingArguments(
    output_dir = '/content',
    save_total_limit = 1,
    evaluation_strategy = "epoch",
    learning_rate= 5e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs =30,
    weight_decay=0.01,
    load_best_model_at_end=True,
    save_strategy = 'epoch',
    metric_for_best_model='pearson',
)

def compute_metrics(p):
    pred, labels = p
    #pred = np.argmax(pred, axis=1)
    #accuracy = accuracy_score(y_true=labels, y_pred=pred)
    #print(pred)

    rmse = np.sqrt(np.mean((pred.reshape(1,-1)[0]-labels)**2))
    pearson = np.corrcoef(pred.reshape(1,-1)[0],labels)[0,1]
    coef, p = spearmanr(pred.reshape(1,-1)[0], labels)

    return {"rmse": rmse,
            "pearson": pearson,
            "spearman" : coef}



class RegressionTrainer(Trainer):
    def compute_loss(self,model,inputs,return_outputs = False):


        bert_ouput = model(**inputs).logits
        batch_size = inputs['input_ids'].size()[0]

        predictions = bert_ouput.view(batch_size)
        predictions_for_loss = bert_ouput.view(batch_size).float()
            
        labels_for_loss = inputs['labels'].float()

        loss = F.mse_loss(predictions_for_loss,labels_for_loss)

        return (loss,(loss,predictions.int())) if return_outputs else loss




trainer = RegressionTrainer(
    model,
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['dev'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

early_stopping = EarlyStoppingCallback(early_stopping_patience = 10)
trainer.add_callback(early_stopping)
trainer.train()

/content/drive/MyDrive/CAMemBERT2/preprocessing2.py:24: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','correct'])
/content/drive/MyDrive/CAMemBERT2/preprocessing2.py:24: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ged = pd.DataFrame(pd.read_csv(f'data/fce-public.{set_type}.original.tsv',sep='  ',names=['word']).word.str.split('\t',1).tolist(),columns = ['word','correct'])
/content/drive/MyDrive/CAMemBERT2/preprocessing2.py:24: ParserWarning: Falling back 

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loadin

Epoch,Training Loss,Validation Loss,Rmse,Pearson,Spearman
1,No log,31.717157,5.631385,0.310726,0.349240
2,No log,29.767151,5.529467,0.434022,0.445650
3,No log,25.302101,4.984977,0.579513,0.591076
4,61.800600,30.348974,5.240468,0.614826,0.604825
5,61.800600,34.421955,5.504544,0.644133,0.650358
6,61.800600,25.730844,4.872115,0.653336,0.647677
7,61.800600,25.572178,4.838647,0.661755,0.671974
8,18.953800,30.086939,5.184110,0.662716,0.670652
9,18.953800,37.039406,5.727128,0.666492,0.670587
10,18.953800,34.241455,5.547522,0.670184,0.675578


***** Running Evaluation *****
  Num examples = 80
  Batch size = 8
Saving model checkpoint to /content/checkpoint-133
Configuration saved in /content/checkpoint-133/config.json
Model weights saved in /content/checkpoint-133/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-133/tokenizer_config.json
Special tokens file saved in /content/checkpoint-133/special_tokens_map.json
Deleting older checkpoint [/content/checkpoint-1596] due to args.save_total_limit
Deleting older checkpoint [/content/checkpoint-2128] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 80
  Batch size = 8
Saving model checkpoint to /content/checkpoint-266
Configuration saved in /content/checkpoint-266/config.json
Model weights saved in /content/checkpoint-266/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-266/tokenizer_config.json
Special tokens file saved in /content/checkpoint-266/special_tokens_map.json
Deleting older checkpoint [/content/checkp

Epoch,Training Loss,Validation Loss,Rmse,Pearson,Spearman
1,No log,31.717157,5.631385,0.310726,0.349240
2,No log,29.767151,5.529467,0.434022,0.445650
3,No log,25.302101,4.984977,0.579513,0.591076
4,61.800600,30.348974,5.240468,0.614826,0.604825
5,61.800600,34.421955,5.504544,0.644133,0.650358
6,61.800600,25.730844,4.872115,0.653336,0.647677
7,61.800600,25.572178,4.838647,0.661755,0.671974
8,18.953800,30.086939,5.184110,0.662716,0.670652
9,18.953800,37.039406,5.727128,0.666492,0.670587
10,18.953800,34.241455,5.547522,0.670184,0.675578


***** Running Evaluation *****
  Num examples = 80
  Batch size = 8
Saving model checkpoint to /content/checkpoint-3325
Configuration saved in /content/checkpoint-3325/config.json
Model weights saved in /content/checkpoint-3325/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-3325/tokenizer_config.json
Special tokens file saved in /content/checkpoint-3325/special_tokens_map.json
Deleting older checkpoint [/content/checkpoint-3192] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 80
  Batch size = 8
Saving model checkpoint to /content/checkpoint-3458
Configuration saved in /content/checkpoint-3458/config.json
Model weights saved in /content/checkpoint-3458/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-3458/tokenizer_config.json
Special tokens file saved in /content/checkpoint-3458/special_tokens_map.json
Deleting older checkpoint [/content/checkpoint-3325] due to args.save_total_limit
***** Running Evaluation *****
 

TrainOutput(global_step=3990, training_loss=22.394099384202693, metrics={'train_runtime': 421.3991, 'train_samples_per_second': 75.534, 'train_steps_per_second': 9.468, 'total_flos': 4216362105047040.0, 'train_loss': 22.394099384202693, 'epoch': 30.0})

In [ ]:
df

,text,tags,answer-s,script_scores,id,set_type,text,tags,answer-s,script_scores,id,set_type,text,tags,answer-s,script_scores,id,set_type
0,"Dear Sir or Madam , I am writing in order to e...","[c, c, c, c, c, c, c, c, c, c, c, c, c, c, i, ...",4.3,31,TR1*0102*2000*01,train,"Dear Mrs Smith , I am sad to read about Richar...","[c, c, c, c, c, c, c, c, c, c, c, c, c, i, i, ...",5.2,30.0,TE1*0100*2001*01,test,13th June 2000 Dear Ms Helen Ryan Competition ...,"[c, c, c, c, c, c, c, c, c, c, c, c, i, c, c, ...",4.3,27.0,TR27*0100*2000*01,dev
1,"Unfortunately , Pat was n't very good at keepi...","[c, c, c, c, c, c, c, c, c, c, c, c, c, c, c, ...",5.1,31,TR1*0102*2000*01,train,Where are you ? is very often the first questi...,"[i, c, c, c, i, c, c, c, c, c, c, c, c, c, c, ...",2.3T,30.0,TE1*0100*2001*01,test,"Dear Kim , Hello ! I hope everything around yo...","[c, c, c, c, c, c, c, c, c, c, c, c, c, c, c, ...",2.3,27.0,TR27*0100*2000*01,dev
2,"10 June 2000 Dear Manager , I would like to co...","[c, c, c, c, c, c, c, c, c, c, c, c, c, c, c, ...",3.3,29,TR2*0102*2000*01,train,Dear Mrs Smith I am writing to reply your lett...,"[c, c, c, c, c, c, c, c, i, c, c, c, i, c, c, ...",3.3,27.0,TE2*0100*2001*01,test,At the begining I would like to thank you . No...,"[i, i, i, c, c, c, c, c, c, c, c, c, c, c, c, ...",3.2,13.0,TR34*0100*2000*01,dev
3,Fashion of the future People will wear this ki...,"[c, c, c, c, c, c, c, c, c, c, c, c, c, i, c, ...",3.3,29,TR2*0102*2000*01,train,Computer Life Now-a-day computer can be found ...,"[c, c, i, i, c, c, c, c, c, c, c, c, c, c, c, ...",3.3,27.0,TE2*0100*2001*01,test,"Lausanne , the 20th of May 2000 Dear M. Smith ...","[c, c, c, c, c, c, c, c, c, c, c, c, c, c, c, ...",3.2,25.0,TR41*0102*2000*01,dev
4,DECEMBER 12TH PRINCIPAL MR . ROBERTSON DEAR SI...,"[c, c, c, c, c, c, c, c, c, c, c, c, i, c, c, ...",2.3,28,TR3*0100*2000*02,train,"Dear Maria First at all , I am sorry about Ric...","[c, c, c, c, c, c, c, c, c, c, i, c, c, c, c, ...",5.1,34.0,TE3*0100*2001*01,test,"Unfortunately , Pat was n't very good at keepi...","[c, c, c, c, c, c, c, c, c, c, c, c, c, c, c, ...",3.1,25.0,TR41*0102*2000*01,dev
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2111,If you ask at twenty womans what is their favo...,"[c, c, c, i, c, i, c, i, i, i, i, c, i, c, c, ...",3.3,29,TR1138*0100*2000*01,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2112,Dear Mr. Smith ; I am writing this letter to m...,"[c, c, c, i, c, c, c, c, c, c, i, c, c, i, c, ...",3.1,24,TR1139*0102*2000*01,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2113,TECHNOLOGICAL ADVANTAGES Technology is the thi...,"[c, c, c, c, c, c, c, c, c, c, c, c, c, c, c, ...",3.2,24,TR1139*0102*2000*01,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2114,"Dear Sir or Madam , I am writing to express my...","[c, c, c, c, c, c, c, c, c, c, c, c, i, c, c, ...",4.3,34,TR1140*0102*2000*01,train,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
dataset = data.get_dataset()
columns_to_drop = list(set(dataset.column_names['train']) - {'attention_mask', 'input_ids', 'answer_scores'})
dataset = dataset.remove_columns(columns_to_drop).rename_column('answer_scores','labels')

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from scipy.stats import spearmanr
import torch
import torch.nn.functional as F
import copy
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

initial_checkpoint = "distilroberta-base"   
tokenizer = AutoTokenizer.from_pretrained(initial_checkpoint, use_fast=True)

num_labels = 1
model = AutoModelForSequenceClassification.from_pretrained('distilroberta-base', num_labels=num_labels)

#freezing layers if needed
modules = [model.roberta.embeddings, *model.roberta.encoder.layer[:6]] #freeze all layers
for module in modules:
    for param in module.parameters():
        param.requires_grad = False

batch_size = 8


args = TrainingArguments(
    output_dir = '/content',
    save_total_limit = 1,
    evaluation_strategy = "epoch",
    learning_rate= 5e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs =30,
    weight_decay=0.01,
    load_best_model_at_end=True,
    save_strategy = 'epoch',
    metric_for_best_model='pearson',
)

def compute_metrics(p):
    pred, labels = p
    #pred = np.argmax(pred, axis=1)
    #accuracy = accuracy_score(y_true=labels, y_pred=pred)
    #print(pred)

    rmse = np.sqrt(np.mean((pred.reshape(1,-1)[0]-labels)**2))
    pearson = np.corrcoef(pred.reshape(1,-1)[0],labels)[0,1]
    coef, p = spearmanr(pred.reshape(1,-1)[0], labels)

    return {"rmse": rmse,
            "pearson": pearson,
            "spearman" : coef}



class RegressionTrainer(Trainer):
    def compute_loss(self,model,inputs,return_outputs = False):


        bert_ouput = model(**inputs).logits
        batch_size = inputs['input_ids'].size()[0]

        predictions = bert_ouput.view(batch_size)
        predictions_for_loss = bert_ouput.view(batch_size).float()
            
        labels_for_loss = inputs['labels'].float()

        loss = F.mse_loss(predictions_for_loss,labels_for_loss)

        return (loss,(loss,predictions.int())) if return_outputs else loss




trainer = RegressionTrainer(
    model,
    args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['dev'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

early_stopping = EarlyStoppingCallback(early_stopping_patience = 10)
trainer.add_callback(early_stopping)
trainer.train()


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.weig

Epoch,Training Loss,Validation Loss,Rmse,Pearson,Spearman
1,No log,7.571424,2.639687,0.400000,0.425421
2,12.228400,6.998272,2.489722,0.502799,0.524829
3,12.228400,6.881902,2.505123,0.485127,0.511125
4,6.211500,7.563689,2.574530,0.488610,0.511889


***** Running Evaluation *****
  Num examples = 156
  Batch size = 8
Saving model checkpoint to /content/checkpoint-264
Configuration saved in /content/checkpoint-264/config.json
Model weights saved in /content/checkpoint-264/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-264/tokenizer_config.json
Special tokens file saved in /content/checkpoint-264/special_tokens_map.json
Deleting older checkpoint [/content/checkpoint-1330] due to args.save_total_limit
Deleting older checkpoint [/content/checkpoint-1463] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 156
  Batch size = 8
Saving model checkpoint to /content/checkpoint-528
Configuration saved in /content/checkpoint-528/config.json
Model weights saved in /content/checkpoint-528/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-528/tokenizer_config.json
Special tokens file saved in /content/checkpoint-528/special_tokens_map.json
Deleting older checkpoint [/content/chec

KeyboardInterrupt: ignored

In [ ]:
orig_data = data.read_data().reset_index()
orig_data['script-s'] = orig_data['script-s'].apply(lambda x : x)
orig_data['text'] = orig_data['text'].str.replace('\n', ' ', regex = True)
orig_data['text'] = orig_data['text'] + '. '
orig_data['text'] = orig_data['text'].str.split().str.join(' ')
dataset_agg = orig_data.groupby('id')[['text']].apply(sum)
dataset_agg['score'] = orig_data.groupby(['id'])['script-s'].max()
dataset_agg['set_type'] = orig_data.groupby(['id'])['set_type'].max()
dataset_agg = dataset_agg.reset_index()
dataset_agg = dataset_agg[['text','score','set_type']]
dataset_dict = {set_type:Dataset.from_pandas(dataset_agg.groupby('set_type').get_group(set_type)) for set_type in ['train','test','dev']}
from transformers import AutoTokenizer

initial_checkpoint = "distilroberta-base"   
tokenizer = AutoTokenizer.from_pretrained(initial_checkpoint, use_fast=True)

def preprocess_function(examples):
    return tokenizer(examples['text'],truncation=True, padding = True, max_length = 512)

encoded_dataset = {set_type : dataset.map(preprocess_function, batched=True) for set_type,dataset in dataset_dict.items()}
columns_to_drop = list(set(encoded_dataset['train'].column_names) - {'attention_mask', 'input_ids', 'score'})
encoded_dataset = {set_type : dataset.remove_columns(columns_to_drop).rename_column('score','labels') for set_type,dataset in encoded_dataset.items()}

import numpy as np
from sklearn.metrics import accuracy_score
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from scipy.stats import spearmanr
import torch
import torch.nn.functional as F
import copy
from transformers import AutoModelForSequenceClassification

num_labels = 1
model = AutoModelForSequenceClassification.from_pretrained('distilroberta-base', num_labels=num_labels)

#freezing layers if needed
modules = [model.roberta.embeddings, *model.roberta.encoder.layer[:6]] #freeze all layers
for module in modules:
    for param in module.parameters():
        param.requires_grad = False

batch_size = 8


args = TrainingArguments(
    output_dir = '/content',
    save_total_limit = 1,
    evaluation_strategy = "epoch",
    learning_rate= 5e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs =30,
    weight_decay=0.01,
    load_best_model_at_end=True,
    save_strategy = 'epoch',
    metric_for_best_model='pearson',
)

def compute_metrics(p):
    pred, labels = p
    #pred = np.argmax(pred, axis=1)
    #accuracy = accuracy_score(y_true=labels, y_pred=pred)
    #print(pred)

    rmse = np.sqrt(np.mean((pred.reshape(1,-1)[0]-labels)**2))
    pearson = np.corrcoef(pred.reshape(1,-1)[0],labels)[0,1]
    coef, p = spearmanr(pred.reshape(1,-1)[0], labels)

    return {"rmse": rmse,
            "pearson": pearson,
            "spearman" : coef}



class RegressionTrainer(Trainer):
    def compute_loss(self,model,inputs,return_outputs = False):


        bert_ouput = model(**inputs).logits
        batch_size = inputs['input_ids'].size()[0]

        predictions = bert_ouput.view(batch_size)
        predictions_for_loss = bert_ouput.view(batch_size).float()
            
        labels_for_loss = inputs['labels'].float()

        loss = F.mse_loss(predictions_for_loss,labels_for_loss)

        return (loss,(loss,predictions.int())) if return_outputs else loss




trainer = RegressionTrainer(
    model,
    args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['dev'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

early_stopping = EarlyStoppingCallback(early_stopping_patience = 10)
trainer.add_callback(early_stopping)
trainer.train()


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loadin

Epoch,Training Loss,Validation Loss,Rmse,Pearson,Spearman
1,No log,31.231716,5.577858,0.375368,0.386648
2,No log,28.776173,5.381682,0.584455,0.601732
3,No log,24.295681,4.925698,0.585002,0.610961
4,62.656800,28.251978,5.122255,0.617135,0.649948
5,62.656800,32.149940,5.382843,0.644795,0.663075
6,62.656800,23.582172,4.703722,0.665119,0.687593
7,62.656800,21.015137,4.486090,0.667204,0.678885
8,19.035600,26.132183,4.861841,0.685630,0.699937
9,19.035600,29.154995,5.138093,0.679567,0.693889
10,19.035600,27.861521,4.947221,0.700041,0.708730


***** Running Evaluation *****
  Num examples = 80
  Batch size = 8
Saving model checkpoint to /content/checkpoint-133
Configuration saved in /content/checkpoint-133/config.json
Model weights saved in /content/checkpoint-133/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-133/tokenizer_config.json
Special tokens file saved in /content/checkpoint-133/special_tokens_map.json
Deleting older checkpoint [/content/checkpoint-665] due to args.save_total_limit
Deleting older checkpoint [/content/checkpoint-798] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 80
  Batch size = 8
Saving model checkpoint to /content/checkpoint-266
Configuration saved in /content/checkpoint-266/config.json
Model weights saved in /content/checkpoint-266/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-266/tokenizer_config.json
Special tokens file saved in /content/checkpoint-266/special_tokens_map.json
Deleting older checkpoint [/content/checkpoi

RuntimeError: ignored

In [ ]:
# orig_data = data.read_data().reset_index()
# orig_data['script-s'] = orig_data['script-s'].apply(lambda x : x)
# orig_data['text'] = orig_data['text'].str.replace('\n', ' ', regex = True)
# orig_data['text'] = orig_data['text'] + '. '
# dataset_agg = orig_data.groupby('id')[['text']].apply(sum)
# dataset_agg['score'] = orig_data.groupby(['id'])['script-s'].max()
# dataset_agg['set_type'] = orig_data.groupby(['id'])['set_type'].max()
# dataset_agg = dataset_agg.reset_index()
# dataset_agg = dataset_agg[['text','score','set_type']]
# dataset_dict = {set_type:Dataset.from_pandas(dataset_agg.groupby('set_type').get_group(set_type)) for set_type in ['train','test','dev']}
# from transformers import AutoTokenizer

# initial_checkpoint = "distilroberta-base"   
# tokenizer = AutoTokenizer.from_pretrained(initial_checkpoint, use_fast=True)

# def preprocess_function(examples):
#     return tokenizer(examples['text'],truncation=True, padding = True, max_length = 512)

# encoded_dataset = {set_type : dataset.map(preprocess_function, batched=True) for set_type,dataset in dataset_dict.items()}
# columns_to_drop = list(set(encoded_dataset['train'].column_names) - {'attention_mask', 'input_ids', 'score'})
# encoded_dataset = {set_type : dataset.remove_columns(columns_to_drop).rename_column('score','labels') for set_type,dataset in encoded_dataset.items()}

In [ ]:
# import numpy as np
# from sklearn.metrics import accuracy_score
# from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
# from scipy.stats import spearmanr
# import torch
# import torch.nn.functional as F
# import copy
# from transformers import AutoModelForSequenceClassification

# num_labels = 1
# model = AutoModelForSequenceClassification.from_pretrained('distilroberta-base', num_labels=num_labels)

# #freezing layers if needed
# modules = [model.roberta.embeddings, *model.roberta.encoder.layer[:6]] #freeze all layers
# for module in modules:
#     for param in module.parameters():
#         param.requires_grad = False

# batch_size = 8


# args = TrainingArguments(
#     output_dir = '/content',
#     save_total_limit = 1,
#     evaluation_strategy = "epoch",
#     learning_rate= 5e-4,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     num_train_epochs =30,
#     weight_decay=0.01,
#     load_best_model_at_end=True,
#     save_strategy = 'epoch',
#     metric_for_best_model='pearson',
# )

# def compute_metrics(p):
#     pred, labels = p
#     #pred = np.argmax(pred, axis=1)
#     #accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     #print(pred)

#     rmse = np.sqrt(np.mean((pred.reshape(1,-1)[0]-labels)**2))
#     pearson = np.corrcoef(pred.reshape(1,-1)[0],labels)[0,1]
#     coef, p = spearmanr(pred.reshape(1,-1)[0], labels)

#     return {"rmse": rmse,
#             "pearson": pearson,
#             "spearman" : coef}



# class RegressionTrainer(Trainer):
#     def compute_loss(self,model,inputs,return_outputs = False):


#         bert_ouput = model(**inputs).logits
#         batch_size = inputs['input_ids'].size()[0]

#         predictions = bert_ouput.view(batch_size)
#         predictions_for_loss = bert_ouput.view(batch_size).float()
            
#         labels_for_loss = inputs['labels'].float()

#         loss = F.mse_loss(predictions_for_loss,labels_for_loss)

#         return (loss,(loss,predictions.int())) if return_outputs else loss




# trainer = RegressionTrainer(
#     model,
#     args,
#     train_dataset=encoded_dataset['train'],
#     eval_dataset=encoded_dataset['dev'],
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

# early_stopping = EarlyStoppingCallback(early_stopping_patience = 10)
# trainer.add_callback(early_stopping)
# trainer.train()


loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loadin

Epoch,Training Loss,Validation Loss,Rmse,Pearson,Spearman
1,No log,32.223347,5.612486,0.433214,0.453495
2,No log,30.467077,5.625833,0.424397,0.448963
3,No log,25.008039,5.076909,0.640758,0.654021
4,62.655900,25.353828,4.833477,0.690154,0.682319
5,62.655900,27.044897,4.930771,0.717483,0.694536


***** Running Evaluation *****
  Num examples = 80
  Batch size = 8
Saving model checkpoint to /content/checkpoint-133
Configuration saved in /content/checkpoint-133/config.json
Model weights saved in /content/checkpoint-133/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-133/tokenizer_config.json
Special tokens file saved in /content/checkpoint-133/special_tokens_map.json
Deleting older checkpoint [/content/checkpoint-1197] due to args.save_total_limit
Deleting older checkpoint [/content/checkpoint-1330] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 80
  Batch size = 8
Saving model checkpoint to /content/checkpoint-266
Configuration saved in /content/checkpoint-266/config.json
Model weights saved in /content/checkpoint-266/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-266/tokenizer_config.json
Special tokens file saved in /content/checkpoint-266/special_tokens_map.json
***** Running Evaluation *****
  Num examp

KeyboardInterrupt: ignored

In [ ]:
run_model(encoded_dataset)

loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loadin

RuntimeError: ignored